In [ ]:
# 基础库
import os                  # 提供与操作系统交互的功能，如文件路径操作
import platform            # 获取系统平台信息
import sys                 # 提供对Python解释器的访问和控制
import urllib.request      # 用于处理URL和网络请求

# 数据处理和科学计算
import numpy as np         # 科学计算库，提供多维数组和矩阵运算
import tensorflow as tf    # 深度学习框架，用于构建和训练神经网络

# 数据可视化
import matplotlib.pyplot as plt          # 绘图库，用于创建静态、动态和交互式可视化
from matplotlib.font_manager import FontProperties, fontManager  # 字体管理，支持中文显示

# Jupyter相关
from IPython.display import clear_output  # 用于清除Jupyter notebook输出

# Kaggle相关
import kagglehub          # Kaggle数据集管理工具，用于下载和访问Kaggle数据集


# 打印Python环境信息
print("Python version:", sys.version)     # 打印Python版本
print("Platform:", platform.platform())   # 打印操作系统平台（完整信息）
print("System:", platform.system())       # 打印操作系统类型（如Windows、Linux）
print("Node:", platform.node())           # 打印网络节点名称（主机名）
print("Release:", platform.release())     # 打印操作系统发行版本
print("Version:", platform.version())     # 打印操作系统版本号（更详细）
print("Machine:", platform.machine())     # 打印计算机类型（如x86_64）
print("Processor:", platform.processor()) # 打印处理器类型

# 检查TensorFlow版本及其依赖
!pip show tensorflow                      # 显示TensorFlow包的详细信息，包括版本和依赖

# 检查系统硬件信息
!cat /proc/cpuinfo | grep 'model name'    # 显示CPU型号（Linux系统）
!cat /proc/meminfo | grep 'MemTotal'      # 显示系统总内存大小（Linux系统）
!df -h                                    # 显示磁盘使用情况（disk free，人类可读格式）
!nvidia-smi                               # 显示NVIDIA GPU信息（如果系统有NVIDIA GPU）


# 数据集



## 数据集获取
 
本项目使用的蘑菇图像数据集托管在 [Kaggle](https://www.kaggle.com/) 平台上，数据集名称为 [huizecai/mushroom](https://www.kaggle.com/datasets/huizecai/mushroom)。该数据集包含了多种常见蘑菇的高清图片，以及对应的分类标签。

为了方便数据获取，我们使用 `kagglehub` 库来自动下载和管理数据集。下面的代码单元格会直接从 Kaggle 下载数据集，并返回保存在本地的路径。数据集下载完成后会被缓存，后续运行时将直接使用缓存版本，无需重复下载。

In [ ]:
# 设置数据集名称
dataset_name = "huizecai/mushroom"  # 指定要下载的Kaggle数据集名称

# 使用KaggleHub下载数据集
path = kagglehub.dataset_download(dataset_name)  # 下载数据集并获取保存路径

# 打印数据集文件的保存路径
print("Path to dataset files:", path)

# 设置数据和标签文件的具体路径
dataset_path = path + '/archive/data'  # 图片数据所在目录的路径
label_path = path + '/archive/label.txt'  # 标签文件的路径

## 数据集类别统计分析
 
为了避免TensorFlow处理中文路径时可能出现的编码问题，本数据集采用了规范化的命名方式:
 - 各蘑菇种类的文件夹以"classXX"格式命名(XX为数字编号)
 - 使用label.txt文件建立文件夹编号与中文名称的映射关系
 - 这种设计既保证了系统兼容性，又方便了数据的管理和使用

In [ ]:
# 获取所有子目录（即蘑菇类别）
# os.listdir() 列出指定目录下的所有文件和文件夹
# os.path.isdir() 判断是否为文件夹
# 使用列表推导式获取所有蘑菇类别的文件夹名
dir_names = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# 读取 label.txt 并解析内容
# 创建一个空字典用于存储类别ID和名称的映射关系
categories = {}
# 以UTF-8编码打开label.txt文件
with open(label_path, 'r', encoding='utf-8') as file:
    for line in file:
        # 去除每行首尾空白字符并按空格分割
        parts = line.strip().split()
        # 确保每行包含两个部分:类别名称和ID
        if len(parts) == 2:
            category_name = parts[0]  # 第一部分为类别名称(中文)
            category_id = parts[1]    # 第二部分为类别ID
            categories[category_id] = category_name  # 建立ID到名称的映射

# 统计每种类别的图像数量
# 创建空字典存储每个类别的图片数量
category_counts = {}
for category_id in categories.keys():
    # 确保目录存在再进行统计
    if category_id in dir_names:
        # 构建完整的类别目录路径
        category_dir = os.path.join(dataset_path, category_id)
        # 统计jpg和jpeg格式的图片数量
        # 使用列表推导式过滤出图片文件并计数
        num_images = len([f for f in os.listdir(category_dir) if f.endswith('.jpg') or f.endswith('.jpeg')])
        # 使用中文类别名称作为键存储图片数量
        category_counts[categories[category_id]] = num_images

# 打印每个类别的图片数量统计结果
print("Category counts:", category_counts)

## 解决matplotlib中文显示问题

matplotlib默认不支持中文字体显示,可能会出现乱码。为了确保数据可视化结果能正确展示中文:
1. 我们将下载并使用"SimHei"(黑体)字体
2. 注册字体到matplotlib的字体管理器
3. 配置全局字体设置

这样可以保证后续所有图表中的中文标题、标签等都能正常显示。

In [ ]:
# 创建img目录(如果不存在)
if not os.path.exists('img'):
    os.makedirs('img')

# 设置字体文件的URL和本地保存路径
font_url = "https://github.com/caijihuize/Mushroom_Classification/raw/main/SimHei.ttf"  # 黑体字体文件的URL
font_name = "SimHei.ttf"  # 本地保存的字体文件名

# 如果字体文件不存在则下载
if not os.path.exists(font_name):
    urllib.request.urlretrieve(font_url, font_name)  # 下载字体文件到本地

# 配置matplotlib的字体设置
fontManager.addfont(font_name)  # 将字体文件添加到matplotlib的字体管理器
font_prop = FontProperties(fname=font_name)  # 创建字体属性对象

# 设置全局字体配置
plt.rcParams['font.family'] = 'SimHei'  # 设置默认字体为黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.rcParams['font.size'] = 20  # 设置全局字体大小
plt.rcParams['axes.titlesize'] = 22  # 设置标题字体大小
plt.rcParams['axes.labelsize'] = 20  # 设置轴标签字体大小
plt.rcParams['figure.dpi'] = 300  # 设置图形DPI为300,提高显示清晰度
plt.rcParams['savefig.dpi'] = 600  # 设置保存图片的DPI为600,提高保存图片的清晰度

# 绘制测试图表验证中文显示
plt.figure(figsize=(10, 6))  # 设置图形大小
plt.title('这是一个标题', fontsize=20)  # 设置标题
plt.xlabel('X轴标签', fontsize=16)  # 设置X轴标签
plt.ylabel('Y轴标签', fontsize=16)  # 设置Y轴标签
plt.plot([0, 1, 2, 3], [0, 1, 4, 9], linewidth=2)  # 绘制简单的折线图,增加线宽提高清晰度

# 保存图表到img目录,使用更高质量的设置
plt.savefig('img/test_plot.png', 
            bbox_inches='tight',  # 自动调整边界
            dpi=800,  # 设置更高的DPI
            format='png',  # 使用PNG格式保存
            facecolor='white',  # 设置白色背景
            edgecolor='none',  # 无边框
            transparent=False)  # 不透明
plt.show()  # 显示图表

## 绘制各种类图片数量的柱状图

In [ ]:
# 准备数据
categories_readable = list(category_counts.keys())  # 获取所有蘑菇种类名称
counts = list(category_counts.values())  # 获取每个种类对应的图片数量

# 创建一个新的图形，设置更大的尺寸以便更好地展示数据
plt.figure(figsize=(18, 16))

# 创建颜色渐变 - 根据数量排序，数量越多颜色越亮
# 首先创建数量和类别的映射关系
count_category_pairs = list(zip(counts, categories_readable))
# 按照数量排序
count_category_pairs.sort(key=lambda x: x[0])
# 提取排序后的类别和数量
sorted_categories = [pair[1] for pair in count_category_pairs]
sorted_counts = [pair[0] for pair in count_category_pairs]

# 创建颜色映射，使用较为柔和的颜色差异
# 使用单一色系的渐变，减小颜色差异
norm = plt.Normalize(min(counts), max(counts))
# 使用Blues色系，颜色差异较小
colors = plt.cm.Blues(norm(counts) * 0.7 + 0.3)  # 缩小颜色范围，增加最小值，减小颜色差异

# 绘制水平柱状图，使用柔和的颜色
bars = plt.barh(categories_readable, counts, color=colors, height=0.7, 
                edgecolor='gray', linewidth=0.5, alpha=0.9)  # 增加透明度使颜色更柔和

# 在每个柱子右侧添加数值标签
for i, bar in enumerate(bars):
    width = bar.get_width()  # 获取柱子的宽度(即图片数量)
    plt.text(width + 2, bar.get_y() + bar.get_height()/2, f'{int(width)}', 
             va='center', ha='left', fontsize=14, fontweight='bold', 
             color='darkblue')  # 美化标签样式

# 设置图表标题和轴标签
plt.title('各蘑菇种类的图片数量分布', fontsize=28, fontweight='bold', pad=20)  # 增大标题字体并加粗
plt.xlabel('图片数量', fontsize=22, labelpad=15)  # 设置x轴标签并增加内边距
plt.ylabel('蘑菇种类', fontsize=22, labelpad=15)  # 设置y轴标签并增加内边距

# 设置坐标轴样式
plt.tick_params(axis='both', which='major', labelsize=16)  # 增大刻度标签字体
plt.xlim(25, max(counts) + max(counts)*0.1)  # 设置x轴范围，留出更多空间

# 添加网格线
plt.grid(axis='x', linestyle='--', alpha=0.4, color='gray')  # 使用更淡的网格线

# 添加背景色
plt.gca().set_facecolor('#f8f9fa')  # 设置浅灰色背景
plt.gca().spines['top'].set_visible(False)  # 移除上边框
plt.gca().spines['right'].set_visible(False)  # 移除右边框
plt.gca().spines['left'].set_linewidth(0.5)  # 减小左边框宽度
plt.gca().spines['bottom'].set_linewidth(0.5)  # 减小下边框宽度

# 自动调整布局，防止标签被截断
plt.tight_layout()

# 保存图表到img目录，使用更高质量的设置
plt.savefig('img/mushroom_distribution.png', 
            bbox_inches='tight', 
            dpi=800, 
            facecolor='#f8f9fa')  # 保存高质量图片，保持背景色一致

# 显示图形
plt.show()

# 训练准备工作

## 加载图像数据集

使用 TensorFlow 的 [image_dataset_from_directory](https://tensorflow.google.cn/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory?hl=en) 函数加载和准备图像数据集：


*   directory=dataset_path ：指定图像数据所在的路径。
*   image_size=(224, 224) ：指定每个图像的大小为224x224像素。
*   batch_size=32 ：指定每个批次包含32张图像。
*   validation_split=0.1 ：指定10%的数据作为验证集。
*   subset='both' ：指定同时返回训练集和验证集。
*   label_mode='categorical' ：指定标签模式为分类模式，返回one-hot编码的标签。
*   seed=21 ：设置随机种子以确保数据集的可重复性。



In [ ]:
import tensorflow as tf

# 加载和准备图像数据集
train_dataset, validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dataset_path,
    image_size=(224, 224),
    batch_size=32,
    validation_split=0.1,
    subset='both',
    label_mode='categorical',
    seed=44
)

## 计算训练集和验证集中各类别图像的分布情况

下面我们将统计训练集和验证集中每个蘑菇类别的图像数量，以便了解数据集的分布特征。

In [ ]:
# 获取类别名称
class_names = train_dataset.class_names
# 从训练数据集中获取所有类别的名称列表

# 初始化字典用于存储每种类别的图像数量
train_category_counts = {name: 0 for name in categories.values()}
validation_category_counts = {name: 0 for name in categories.values()}
# 创建两个字典，分别用于存储训练集和验证集中每个类别的图像数量
# 使用字典推导式初始化，键为类别的中文名称，值初始化为0

# 统计训练集中的图像数量
for images, labels in train_dataset:
    # 遍历训练数据集中的每个批次，每个批次包含图像和对应的标签
    for label in labels.numpy():
        # 将标签张量转换为numpy数组并遍历
        category_name = class_names[np.argmax(label)]
        # np.argmax(label)找出one-hot编码中值为1的索引位置
        # 通过索引从class_names中获取对应的类别名称
        train_category_counts[categories[category_name]] += 1
        # 将该类别在训练集中的计数加1，使用categories字典将英文名映射为中文名

# 统计验证集中的图像数量
for images, labels in validation_dataset:
    # 遍历验证数据集中的每个批次
    for label in labels.numpy():
        # 同样处理验证集中的标签
        category_name = class_names[np.argmax(label)]
        validation_category_counts[categories[category_name]] += 1
        # 将该类别在验证集中的计数加1

# 打印统计结果
print("训练集类别图像数量统计:", train_category_counts)
print("验证集类别图像数量统计:", validation_category_counts)
# 输出训练集和验证集中各个类别的图像数量统计结果

## 绘制训练集和验证集的柱状图

In [ ]:
# 准备数据
categories_readable = list(categories.values())  # 获取所有蘑菇类别的中文名称列表
train_counts = [train_category_counts[name] for name in categories_readable]  # 获取每个类别在训练集中的图片数量
validation_counts = [validation_category_counts[name] for name in categories_readable]  # 获取每个类别在验证集中的图片数量

# 创建一个新的图形
plt.figure(figsize=(14, 8))  # 设置图形大小为14x8英寸

# 设置柱子的位置
bar_width = 0.35  # 设置柱子的宽度
index = np.arange(len(categories_readable))  # 创建类别索引数组

# 绘制训练集柱状图
bars_train = plt.barh(index + bar_width, train_counts, bar_width, 
                     label='训练集', color='skyblue')  # 绘制训练集的水平柱状图

# 绘制验证集柱状图
bars_validation = plt.barh(index, validation_counts, bar_width, 
                         label='测试集', color='orange')  # 绘制验证集的水平柱状图

# 添加数据标签的函数
def add_labels(bars):
    """为每个柱子添加数值标签"""
    for bar in bars:
        width = bar.get_width()  # 获取柱子的宽度(即数值)
        plt.text(width + 1,  # x坐标(数值右侧)
                bar.get_y() + bar.get_height() / 2,  # y坐标(柱子中间)
                '%d' % int(width),  # 显示的文本(整数格式)
                ha='left', va='center')  # 文本对齐方式

# 为训练集和验证集的柱子添加标签
add_labels(bars_train)
add_labels(bars_validation)

# 设置图表标题和轴标签
plt.title('各蘑菇种类的图片数量 (Train vs Validation)', fontsize=20)
plt.xlabel('图片数量', fontsize=16)
plt.ylabel('蘑菇种类', fontsize=16)

# 设置Y轴刻度标签为蘑菇类别名称
plt.yticks(index + bar_width / 2, categories_readable)

# 显示网格线
plt.grid(axis='x', linestyle='--', alpha=0.7)  # 只显示x轴方向的网格线

# 添加图例
plt.legend()

# 自动调整布局，防止标签被截断
plt.tight_layout()

# 保存图形
plt.savefig('img/mushroom_distribution_vs.png', dpi=800, bbox_inches='tight')

# 显示图形
plt.show()

## 显示数据集中的图像样本

In [ ]:
# 获取训练数据集中的类别名称
class_names = train_dataset.class_names

# 设置要在图中显示的随机样本图像数量 
num_images_to_show = 4

# 初始化存储图像和标签的列表
images_to_display = []  # 用于存储待显示的图像
labels_to_display = []  # 用于存储对应的标签

# 从训练数据集中随机抽取一批数据
for images, labels in train_dataset.take(1):
    # 随机选择指定数量的图像索引,不重复
    indices = np.random.choice(range(images.shape[0]), 
                             num_images_to_show, 
                             replace=False)
    
    # 根据随机索引获取对应的图像和标签
    for index in indices:
        images_to_display.append(images[index])
        labels_to_display.append(labels[index])

# 创建1行num_images_to_show列的子图网格
fig, axes = plt.subplots(1, num_images_to_show, figsize=(12, 4))

# 遍历显示每张图像
for i, (image, label) in enumerate(zip(images_to_display, labels_to_display)):
    ax = axes[i]
    # 将图像数据转换为uint8类型并显示
    ax.imshow(image.numpy().astype("uint8"))
    # 设置图像标题为对应的蘑菇类别名称
    ax.set_title(categories[class_names[np.argmax(label.numpy())]], 
                fontsize=20)
    # 关闭坐标轴显示
    ax.axis("off")

# 自动调整子图之间的间距
plt.tight_layout()

# 保存图形
plt.savefig('img/mushroom_samples.png', dpi=800, bbox_inches='tight')

# 显示整个图形
plt.show()

# 数据增强

数据增强是一种常用的技术，用于通过变换现有数据来增加训练数据的多样性，从而提高模型的泛化能力。在图像处理中，常见的数据增强技术包括旋转、翻转、缩放、裁剪、亮度调整等。

## 定义数据增强层

In [10]:
# 定义数据增强的预处理层
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),  # 水平翻转
    tf.keras.layers.RandomRotation(0.2),       # 随机旋转最多20%
    tf.keras.layers.RandomZoom(0.2, 0.2),      # 随机缩放
    tf.keras.layers.RandomContrast(0.2),       # 随机对比度调整
    tf.keras.layers.RandomBrightness(0.2)      # 随机亮度调整
])

## 定义显示数据增强示例函数

In [11]:
def demo_augmentation(sample_image, model, num_aug):
    '''接收单个图像数组，然后使用模型生成num_aug个数据增强变换'''

    # 初始化预览图像列表
    image_preview = []

    # 将输入图像转换为PIL图像实例
    sample_image_pil = tf.keras.utils.array_to_img(sample_image)

    # 将原始图像添加到列表中
    image_preview.append(sample_image_pil)

    # 应用图像增强并将结果添加到列表中
    for i in range(NUM_AUG):
        # 扩展维度以适应模型输入
        sample_image_aug = model(tf.expand_dims(sample_image, axis=0))
        # 将增强后的图像转换为PIL格式
        sample_image_aug_pil = tf.keras.utils.array_to_img(tf.squeeze(sample_image_aug))
        image_preview.append(sample_image_aug_pil)

    # 创建子图布局
    fig, axes = plt.subplots(1, NUM_AUG + 1, figsize=(12, 12))

    # 显示所有图像
    for index, ax in enumerate(axes):
        ax.imshow(image_preview[index])
        ax.set_axis_off()

        # 设置图像标题
        if index == 0:
            ax.set_title('原始图像')
        else:
            ax.set_title(f'增强样本 {index}')

## 获取随机批次

In [ ]:
# 获取训练集中随机的一批次图片
sample_batch = list(train_dataset.take(1))[0][0]
print(f'images per batch: {len(sample_batch)}')

## 显示数据增强示例

In [ ]:
# 设置每个样本的数据增强次数
NUM_AUG = 5

# 对训练集中前4张图片进行数据增强演示
# 使用demo_augmentation函数对每张图片进行NUM_AUG次数据增强
# 并将原始图片和增强后的图片并排显示
demo_augmentation(sample_batch[0], data_augmentation, NUM_AUG)  # 处理第1张图片
demo_augmentation(sample_batch[1], data_augmentation, NUM_AUG)  # 处理第2张图片 
demo_augmentation(sample_batch[2], data_augmentation, NUM_AUG)  # 处理第3张图片
demo_augmentation(sample_batch[3], data_augmentation, NUM_AUG)  # 处理第4张图片

# 模型训练相关函数定义
以下定义了用于模型训练的核心函数,包括:
- train_model: 训练深度学习模型的主函数,负责模型编译、配置回调函数和训练过程
- 回调函数:
  - early_stopping: 提前停止训练,防止过拟合
  - lr_scheduler: 动态调整学习率,优化训练效果
- 训练参数:
  - optimizer: adam优化器
  - loss: 分类交叉熵损失函数
  - metrics: 准确率评估指标

In [14]:
# 训练模型函数
def train_model(model, train_dataset=train_dataset, validation_dataset=validation_dataset, epochs=30):
    """
    训练深度学习模型的函数
    
    参数:
        model: 待训练的模型
        train_dataset: 训练数据集
        validation_dataset: 验证数据集  
        epochs: 训练轮数,默认30轮
        
    返回:
        history: 训练历史记录,包含loss和accuracy等指标
    """
    # 配置模型训练参数
    # 使用adam优化器,交叉熵损失函数,准确率评估指标
    model.compile(optimizer='adam', 
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    # 配置回调函数
    # early_stopping: 当验证集loss在6轮内没有改善时停止训练,并恢复最佳权重
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=6, 
        restore_best_weights=True
    )
    
    # lr_scheduler: 当验证集loss在3轮内没有改善时,学习率减半,最小到1e-7
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7
    )

    # 开始训练
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=validation_dataset, 
        callbacks=[early_stopping, lr_scheduler]
    )

    return history

# 模型量化和保存
 
为了在移动设备和嵌入式系统上高效部署模型,我们需要:
1. 将训练好的Keras模型量化 - 把权重从float32转为int8,显著减小模型大小
2. 转换为TFLite格式 - TFLite是专门为移动和嵌入式设备优化的轻量级框架
3. 保存模型到文件 - 以便后续部署使用
下面两个函数实现了这些功能:
- quantize_and_convert_to_tflite(): 模型量化和TFLite转换
- save_model(): 保存模型到文件



In [15]:
def quantize_and_convert_to_tflite(model):
    """
    将Keras模型量化并转换为TFLite格式
    
    参数:
        model: Keras模型对象
        
    返回:
        tflite_quant_model: 量化后的TFLite模型
        
    说明:
        - 使用TFLite转换器将Keras模型转换为TFLite格式
        - 启用默认优化以减小模型大小和提高推理性能
        - 量化可以将模型权重从float32转换为int8,显著减小模型大小
    """
    # 创建TFLite转换器
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    # 启用默认优化(包括权重量化)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # 执行转换
    tflite_quant_model = converter.convert()
    
    return tflite_quant_model

def save_model(model, output_path):
    """
    保存模型到文件
    
    参数:
        model: 要保存的模型对象
        output_path: 保存模型的文件路径
        
    说明:
        - 以二进制格式写入模型到指定路径
        - 通常用于保存TFLite模型(.tflite文件)
    """
    with open(output_path, 'wb') as f:
        f.write(model)

可视化训练历史

下面的函数用于可视化模型训练过程中的指标变化:

- show_history(): 绘制训练历史中的损失和准确率曲线
  - 创建1x2的子图布局
  - 左图显示训练和验证损失
  - 右图显示训练和验证准确率
  - 使用matplotlib进行绘图



In [16]:
def show_history(history):
    """
    可视化模型训练历史中的损失和准确率曲线
    
    参数:
        history: 模型训练返回的History对象,包含训练过程中的指标数据
        
    功能:
        - 创建一个1x2的子图布局
        - 左图显示训练集和验证集的损失曲线
        - 右图显示训练集和验证集的准确率曲线
        - 自动调整布局并显示图形
        
    返回:
        无返回值,直接显示图形
    """
    # 创建12x4英寸的画布
    plt.figure(figsize=(12, 4))

    # 绘制损失曲线
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # 绘制准确率曲线
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # 自动调整子图布局
    plt.tight_layout()
    # 显示图形
    plt.show()

In [17]:
def combine_history(histories, labels):
    """
    将多个模型的训练历史进行对比可视化并保存图片
    
    参数:
        histories: 列表，包含多个模型训练返回的History对象
        labels: 列表，每个模型对应的标签名称
        
    功能:
        - 找出所有模型中最小的训练轮数
        - 创建1x2的子图布局
        - 左图对比各模型的训练集和验证集损失曲线
        - 右图对比各模型的训练集和验证集准确率曲线
        - 使用不同线型区分训练集和验证集
        - 添加图例和网格线增强可读性
        - 保存图片到本地
    """
    # 找出所有模型中最小的训练轮数,用于对齐显示
    min_epochs = min([len(history.history['loss']) for history in histories])
    epochs = range(1, min_epochs + 1)

    # 创建14x6英寸的画布
    plt.figure(figsize=(14, 6))

    # 绘制损失曲线对比图
    plt.subplot(1, 2, 1)
    for i, history in enumerate(histories):
        # 截取相同长度的训练历史数据
        train_loss = history.history['loss'][:min_epochs]
        val_loss = history.history['val_loss'][:min_epochs]
        # 使用虚线表示训练集,实线表示验证集
        plt.plot(epochs, train_loss, label=f'{labels[i]} Train Loss', linestyle='--')
        plt.plot(epochs, val_loss, label=f'{labels[i]} Val Loss')

    plt.title('Training and Validation Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # 绘制准确率曲线对比图
    plt.subplot(1, 2, 2)
    for i, history in enumerate(histories):
        # 截取相同长度的训练历史数据
        train_acc = history.history['accuracy'][:min_epochs]
        val_acc = history.history['val_accuracy'][:min_epochs]
        # 使用虚线表示训练集,实线表示验证集
        plt.plot(epochs, train_acc, label=f'{labels[i]} Train Accuracy', linestyle='--')
        plt.plot(epochs, val_acc, label=f'{labels[i]} Val Accuracy')

    plt.title('Training and Validation Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # 自动调整子图布局
    plt.tight_layout()
    
    # 保存图片
    plt.savefig('img/model_comparison.png', dpi=1000, bbox_inches='tight')
    
    # 显示图形
    plt.show()

In [18]:
from IPython.display import clear_output

def calculate_class_accuracies(model, dataset):
    """
    计算并可视化每个类别的准确率
    
    参数:
        model: 训练好的模型
        dataset: 数据集
        
    返回:
        dict: 每个类别的准确率字典
    """
    # 收集所有预测结果和真实标签
    all_predictions = []
    all_true_labels = []

    for images, labels in dataset:
        predictions = model.predict(images)
        predicted_classes = np.argmax(predictions, axis=1) 
        true_classes = np.argmax(labels, axis=1)
        
        all_predictions.extend(predicted_classes)
        all_true_labels.extend(true_classes)

    # 计算每个类别的准确率
    class_accuracies = {}
    for class_index in range(len(class_names)):
        mask = (np.array(all_true_labels) == class_index)
        correct_predictions = np.sum(np.array(all_predictions)[mask] == class_index)
        total_samples = np.sum(mask)
        if total_samples > 0:
            accuracy = correct_predictions / total_samples
        else:
            accuracy = 0
        class_accuracies[class_names[class_index]] = accuracy

    clear_output(wait=True)

    # 准备绘图数据
    categories_readable = list(categories[class_name] for class_name in class_accuracies.keys())
    accuracies = list(class_accuracies.values())

    # 创建水平柱状图
    plt.figure(figsize=(14, 8))
    bars = plt.barh(categories_readable, accuracies, color='skyblue')

    # 在柱状图上添加数值标签
    for bar in bars:
        width = bar.get_width()
        plt.text(width + 0.01, bar.get_y() + bar.get_height()/2, f'{width:.4f}', va='center')

    # 设置图表标题和标签
    plt.title('各类别准确率', fontsize=20)
    plt.xlabel('准确率', fontsize=16)
    plt.ylabel('类别', fontsize=16)
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # 保存图片
    plt.savefig('class_accuracies.png', dpi=800, bbox_inches='tight')
    plt.show()

    return class_accuracies

In [32]:
def evaluate_model(model, dataset):
    """使用TensorFlow内置metrics评估Keras模型在数据集上的性能
    
    Args:
        model: Keras模型对象
        dataset: tf.data.Dataset数据集对象
        
    Returns:
        tuple: (top1准确率, top5准确率, 模型大小, 推理速度)
    """
    # 计算模型大小 (MB)
    model_size_mb = sum(np.prod(w.shape) for w in model.get_weights()) * 4 / (1024 * 1024)  

    # 创建评估指标
    top1_accuracy = tf.keras.metrics.CategoricalAccuracy()  # Top-1准确率评估器
    top5_accuracy = tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top5_accuracy')  # Top-5准确率评估器

    # 创建损失函数和平均损失计算器
    loss_object = tf.keras.losses.CategoricalCrossentropy()  # 分类交叉熵损失函数
    loss_avg = tf.keras.metrics.Mean()  # 用于计算平均损失

    # 记录推理时间
    total_time = 0
    total_samples = 0

    # 在数据集上运行评估
    for images, labels in dataset:
        start_time = time.time()
        predictions = model(images, training=False)  # 获取模型预测结果
        end_time = time.time()
        
        # 累计推理时间和样本数
        total_time += end_time - start_time
        total_samples += len(images)
        
        loss = loss_object(labels, predictions)  # 计算损失值

        # 更新评估指标
        loss_avg.update_state(loss)
        top1_accuracy.update_state(labels, predictions)
        top5_accuracy.update_state(labels, predictions)

    # 计算平均推理速度 (ms/sample)
    avg_inference_time = (total_time / total_samples) * 1000

    # 获取最终评估结果
    loss = loss_avg.result().numpy()
    top1 = top1_accuracy.result().numpy()
    top5 = top5_accuracy.result().numpy()

    # 打印评估结果
    print(f"Loss: {loss:.4f}")
    print(f"Top-1 Accuracy: {top1:.4f}")
    print(f"Top-5 Accuracy: {top5:.4f}")
    print(f"Model Size: {model_size_mb:.2f} MB")
    print(f"Average Inference Time: {avg_inference_time:.2f} ms/sample")

    return top1, top5, model_size_mb, avg_inference_time

def evaluate_tflite_model(tflite_model, dataset):
    """评估TFLite模型的性能
    
    Args:
        tflite_model: TFLite模型的二进制内容
        dataset: tf.data.Dataset数据集对象
        
    Returns:
        tuple: (top1准确率, top5准确率, 模型大小, 推理速度)
    """
    # 计算TFLite模型大小 (MB)
    model_size_mb = len(tflite_model) / (1024 * 1024)

    # 创建并初始化TFLite解释器
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    # 获取模型输入输出张量的详细信息
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # 创建评估指标
    top1_accuracy = tf.keras.metrics.Accuracy()  # Top-1准确率评估器
    top5_accuracy = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)  # Top-5准确率评估器

    total_samples = 0  # 样本计数器
    total_time = 0  # 总推理时间

    # 遍历数据集进行评估
    for images, labels in dataset:
        # 将one-hot标签转换为类别索引
        true_labels = tf.argmax(labels, axis=1).numpy()

        batch_size = images.shape[0]
        # 存储批次预测结果的数组
        batch_predictions = np.zeros((batch_size, len(class_names)), dtype=np.float32)

        # 对批次中的每个样本进行推理
        for i in range(batch_size):
            total_samples += 1

            # 预处理输入数据
            input_data = np.expand_dims(images[i].numpy(), axis=0).astype(np.float32)
            
            # 记录推理时间
            start_time = time.time()
            
            # 设置输入数据并运行推理
            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()

            # 获取预测结果
            batch_predictions[i] = interpreter.get_tensor(output_details[0]['index'])[0]
            
            end_time = time.time()
            total_time += end_time - start_time

        # 更新评估指标
        top1_accuracy.update_state(
            true_labels,
            np.argmax(batch_predictions, axis=1)
        )
        top5_accuracy.update_state(
            true_labels,
            batch_predictions
        )

    # 计算平均推理速度 (ms/sample)
    avg_inference_time = (total_time / total_samples) * 1000

    # 获取最终评估结果
    top1 = top1_accuracy.result().numpy()
    top5 = top5_accuracy.result().numpy()

    # 打印评估结果
    print(f"TFLite模型 Top-1 准确率: {top1:.4f}")
    print(f"TFLite模型 Top-5 准确率: {top5:.4f}")
    print(f"TFLite模型大小: {model_size_mb:.2f} MB")
    print(f"TFLite模型平均推理时间: {avg_inference_time:.2f} ms/sample")

    return top1, top5, model_size_mb, avg_inference_time

# MobileNetV1模型

In [19]:
def get_MobileNetV1():
  """
  构建并返回一个基于MobileNetV1的迁移学习模型
  
  模型架构:
  1. 使用预训练的MobileNetV1作为特征提取器
  2. 添加数据增强层进行训练时的数据增强
  3. 添加预处理层对输入进行标准化
  4. 添加自定义分类器头部进行具体任务的分类
  
  返回:
  - 构建好的MobileNetV1模型
  """
  # 初始化基础模型 - 使用ImageNet预训练权重,不包含顶层分类器
  pre_trained_model = tf.keras.applications.MobileNet(
      input_shape=(224, 224, 3),  # 输入图像尺寸为224x224,3通道
      include_top=False,  # 不包含顶层分类器
      weights='imagenet'  # 加载ImageNet预训练权重
  )

  # 冻结基础模型的权重 - 固定特征提取器,只训练新添加的分类层
  for layer in pre_trained_model.layers:
      layer.trainable = False

  # 应用数据增强和预处理
  x = data_augmentation(pre_trained_model.input)  # 添加数据增强提高模型泛化能力
  x = tf.keras.applications.mobilenet.preprocess_input(x)  # MobileNet专用的预处理方法

  # 添加自定义顶层分类器
  x = pre_trained_model(x)  # 通过预训练模型提取特征
  x = tf.keras.layers.GlobalAveragePooling2D()(x)  # 全局平均池化减少参数量
  x = tf.keras.layers.Dropout(0.5)(x)  # dropout层防止过拟合
  x = tf.keras.layers.Dense(128, activation='relu')(x)  # 全连接层
  x = tf.keras.layers.Dense(256, activation='relu')(x)  # 全连接层增加非线性
  predictions = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)  # 输出层,使用softmax进行多分类

  # 构建最终模型 - 将输入和输出组装成完整的模型
  MobileNetV1 = tf.keras.models.Model(inputs=pre_trained_model.input, outputs=predictions)

  return MobileNetV1

# MobileNetV2模型

In [20]:
def get_MobileNetV2():
  """
  构建并返回一个基于MobileNetV2的迁移学习模型
  
  模型架构:
  1. 使用预训练的MobileNetV2作为特征提取器
  2. 添加数据增强层进行训练时的数据增强
  3. 添加预处理层对输入进行标准化
  4. 添加自定义分类器头部进行具体任务的分类
  
  返回:
  - 构建好的MobileNetV2模型
  """
  # 初始化基础模型 - 使用ImageNet预训练权重,不包含顶层分类器
  pre_trained_model = tf.keras.applications.MobileNetV2(
      input_shape=(224, 224, 3),  # 输入图像尺寸为224x224,3通道
      include_top=False,  # 不包含顶层分类器
      weights='imagenet'  # 加载ImageNet预训练权重
  )

  # 冻结基础模型的权重 - 固定特征提取器,只训练新添加的分类层
  for layer in pre_trained_model.layers:
      layer.trainable = False

  # 应用数据增强和预处理
  x = data_augmentation(pre_trained_model.input)  # 添加数据增强提高模型泛化能力
  x = tf.keras.applications.mobilenet_v2.preprocess_input(x)  # MobileNetV2专用的预处理方法

  # 添加自定义顶层分类器
  x = pre_trained_model(x)  # 通过预训练模型提取特征
  x = tf.keras.layers.GlobalAveragePooling2D()(x)  # 全局平均池化减少参数量
  x = tf.keras.layers.Dropout(0.5)(x)  # dropout层防止过拟合
  x = tf.keras.layers.Dense(128, activation='relu')(x)  # 全连接层
  x = tf.keras.layers.Dense(256, activation='relu')(x)  # 全连接层增加非线性
  predictions = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)  # 输出层,使用softmax进行多分类

  # 构建最终模型 - 将输入和输出组装成完整的模型
  MobileNetV2 = tf.keras.models.Model(inputs=pre_trained_model.input, outputs=predictions)

  return MobileNetV2

# EfficientNet模型

In [21]:
def get_EfficientNetB0():
  """
  构建并返回一个基于EfficientNetB0的迁移学习模型
  
  模型架构:
  1. 使用预训练的EfficientNetB0作为特征提取器,该模型在ImageNet数据集上预训练
  2. 添加数据增强层提高模型泛化能力
  3. 添加预处理层对输入进行标准化
  4. 添加自定义分类器头部进行具体任务的分类
  
  返回:
  - 构建好的EfficientNetB0模型
  """
  # 初始化基础模型 - 使用ImageNet预训练权重,不包含顶层分类器
  pre_trained_model = tf.keras.applications.EfficientNetB0(
      input_shape=(224, 224, 3),  # 输入图像尺寸为224x224,3通道
      include_top=False,  # 不包含顶层分类器
      weights='imagenet'  # 加载ImageNet预训练权重
  )

  # 冻结基础模型的权重 - 固定特征提取器,只训练新添加的分类层
  for layer in pre_trained_model.layers:
      layer.trainable = False

  # 应用数据增强和预处理
  x = data_augmentation(pre_trained_model.input)  # 添加数据增强提高模型泛化能力 
  x = tf.keras.applications.efficientnet.preprocess_input(x)  # EfficientNet专用的预处理方法

  # 添加自定义顶层分类器
  x = pre_trained_model(x)  # 通过预训练模型提取特征
  x = tf.keras.layers.GlobalAveragePooling2D()(x)  # 全局平均池化减少参数量
  x = tf.keras.layers.Dropout(0.5)(x)  # dropout层防止过拟合
  x = tf.keras.layers.Dense(128, activation='relu')(x)  # 全连接层
  x = tf.keras.layers.Dense(256, activation='relu')(x)  # 全连接层增加非线性
  predictions = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)  # 输出层,使用softmax进行多分类

  # 构建最终模型 - 将输入和输出组装成完整的模型
  EfficientNetB0 = tf.keras.models.Model(inputs=pre_trained_model.input, outputs=predictions)

  return EfficientNetB0

# ResNet模型

In [22]:
def get_ResNet101():
  """
  构建并返回一个基于ResNet101的迁移学习模型
  
  模型架构:
  1. 使用预训练的ResNet101作为特征提取器,该模型在ImageNet数据集上预训练
  2. 添加数据增强层提高模型泛化能力
  3. 添加预处理层对输入进行标准化
  4. 添加自定义分类器头部进行具体任务的分类
  
  返回:
  - 构建好的ResNet101模型
  """
  # 初始化基础模型 - 使用ImageNet预训练权重,不包含顶层分类器
  pre_trained_model = tf.keras.applications.ResNet101(
      input_shape=(224, 224, 3),  # 输入图像尺寸为224x224,3通道
      include_top=False,  # 不包含顶层分类器
      weights='imagenet'  # 加载ImageNet预训练权重
  )

  # 冻结基础模型的权重 - 固定特征提取器,只训练新添加的分类层
  for layer in pre_trained_model.layers:
      layer.trainable = False

  # 应用数据增强和预处理
  x = data_augmentation(pre_trained_model.input)  # 添加数据增强提高模型泛化能力
  x = tf.keras.applications.resnet.preprocess_input(x)  # ResNet专用的预处理方法

  # 添加自定义顶层分类器
  x = pre_trained_model(x)  # 通过预训练模型提取特征
  x = tf.keras.layers.GlobalAveragePooling2D()(x)  # 全局平均池化减少参数量
  x = tf.keras.layers.Dropout(0.5)(x)  # dropout层防止过拟合
  x = tf.keras.layers.Dense(128, activation='relu')(x)  # 全连接层
  x = tf.keras.layers.Dense(256, activation='relu')(x)  # 全连接层增加非线性
  predictions = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)  # 输出层,使用softmax进行多分类

  # 构建最终模型 - 将输入和输出组装成完整的模型
  ResNet101 = tf.keras.models.Model(inputs=pre_trained_model.input, outputs=predictions)

  return ResNet101

# 训练

In [ ]:
# 获取MobileNetV1模型
MobileNetV1 = get_MobileNetV1()

# 训练MobileNetV1模型并保存训练历史
history_MobileNetV1 = train_model(MobileNetV1)

# 将模型量化并转换为TFLite格式以便部署
MobileNetV1_tflite = quantize_and_convert_to_tflite(MobileNetV1)

In [ ]:
# 获取MobileNetV2模型
MobileNetV2 = get_MobileNetV2()

# 训练MobileNetV2模型并保存训练历史
history_MobileNetV2 = train_model(MobileNetV2)

# 将模型量化并转换为TFLite格式以便部署
MobileNetV2_tflite = quantize_and_convert_to_tflite(MobileNetV2)

In [ ]:
# 获取EfficientNetB0模型
EfficientNetB0 = get_EfficientNetB0()

# 训练EfficientNetB0模型并保存训练历史
history_EfficientNetB0 = train_model(EfficientNetB0)

# 将模型量化并转换为TFLite格式以便部署
EfficientNetB0_tflite = quantize_and_convert_to_tflite(EfficientNetB0)

In [ ]:
# 获取ResNet101模型
ResNet101 = get_ResNet101()

# 训练ResNet101模型并保存训练历史
history_ResNet101 = train_model(ResNet101)

# 将模型量化并转换为TFLite格式以便部署
ResNet101_tflite = quantize_and_convert_to_tflite(ResNet101)

In [ ]:
# 合并所有模型的训练历史数据并可视化比较

# 创建一个2x2的子图布局,设置更大的图像尺寸和DPI以提高清晰度
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10), dpi=1000)

# 绘制训练准确率对比
ax1.plot(history_MobileNetV1.history['accuracy'], label='MobileNetV1', linewidth=2)
ax1.plot(history_MobileNetV2.history['accuracy'], label='MobileNetV2', linewidth=2)
ax1.plot(history_EfficientNetB0.history['accuracy'], label='EfficientNetB0', linewidth=2)
ax1.plot(history_ResNet101.history['accuracy'], label='ResNet101', linewidth=2)
ax1.set_title('训练准确率对比', fontsize=12, pad=10)
ax1.set_xlabel('轮次', fontsize=10)
ax1.set_ylabel('准确率', fontsize=10)
ax1.legend(fontsize=9)
ax1.grid(True, linestyle='--', alpha=0.7)

# 绘制验证准确率对比
ax2.plot(history_MobileNetV1.history['val_accuracy'], label='MobileNetV1', linewidth=2)
ax2.plot(history_MobileNetV2.history['val_accuracy'], label='MobileNetV2', linewidth=2)
ax2.plot(history_EfficientNetB0.history['val_accuracy'], label='EfficientNetB0', linewidth=2)
ax2.plot(history_ResNet101.history['val_accuracy'], label='ResNet101', linewidth=2)
ax2.set_title('验证准确率对比', fontsize=12, pad=10)
ax2.set_xlabel('轮次', fontsize=10)
ax2.set_ylabel('准确率', fontsize=10)
ax2.legend(fontsize=9)
ax2.grid(True, linestyle='--', alpha=0.7)

# 绘制训练损失对比
ax3.plot(history_MobileNetV1.history['loss'], label='MobileNetV1', linewidth=2)
ax3.plot(history_MobileNetV2.history['loss'], label='MobileNetV2', linewidth=2)
ax3.plot(history_EfficientNetB0.history['loss'], label='EfficientNetB0', linewidth=2)
ax3.plot(history_ResNet101.history['loss'], label='ResNet101', linewidth=2)
ax3.set_title('训练损失对比', fontsize=12, pad=10)
ax3.set_xlabel('轮次', fontsize=10)
ax3.set_ylabel('损失', fontsize=10)
ax3.legend(fontsize=9)
ax3.grid(True, linestyle='--', alpha=0.7)

# 绘制验证损失对比
ax4.plot(history_MobileNetV1.history['val_loss'], label='MobileNetV1', linewidth=2)
ax4.plot(history_MobileNetV2.history['val_loss'], label='MobileNetV2', linewidth=2)
ax4.plot(history_EfficientNetB0.history['val_loss'], label='EfficientNetB0', linewidth=2)
ax4.plot(history_ResNet101.history['val_loss'], label='ResNet101', linewidth=2)
ax4.set_title('验证损失对比', fontsize=12, pad=10)
ax4.set_xlabel('轮次', fontsize=10)
ax4.set_ylabel('损失', fontsize=10)
ax4.legend(fontsize=9)
ax4.grid(True, linestyle='--', alpha=0.7)

# 调整子图布局
plt.tight_layout(pad=3.0)

# 保存高清图片
plt.savefig('model_comparison.png', dpi=1000, bbox_inches='tight')
plt.show()


# 评估

本节将对训练好的模型进行评估,包括:
1. 评估原始模型在验证集上的性能指标(准确率等)
2. 评估量化后的TFLite模型在验证集上的性能指标
3. 将量化后的TFLite模型保存到文件中以便部署

我们将依次评估以下4个模型:
- MobileNetV1: 轻量级CNN模型,适合移动端部署
- MobileNetV2: MobileNetV1的改进版本,性能更好
- EfficientNetB0: 高效CNN模型,在准确率和效率之间取得平衡
- ResNet101: 经典的深度残差网络,性能强大但计算量较大

In [ ]:
# 评估并保存MobileNetV1模型
print("MobileNetV1:\n")
# 评估原始模型在验证集上的性能
evaluate_model(MobileNetV1, validation_dataset)
# 评估量化后的TFLite模型在验证集上的性能
evaluate_tflite_model(MobileNetV1_tflite, validation_dataset)
# 将量化后的TFLite模型保存到文件
save_model(MobileNetV1_tflite, "MobileNetV1.tflite")

# 评估并保存MobileNetV2模型
print("\nMobileNetV2:\n") 
# 评估原始模型在验证集上的性能
evaluate_model(MobileNetV2, validation_dataset)
# 评估量化后的TFLite模型在验证集上的性能
evaluate_tflite_model(MobileNetV2_tflite, validation_dataset)
# 将量化后的TFLite模型保存到文件
save_model(MobileNetV2_tflite, "MobileNetV2.tflite")

# 评估并保存EfficientNetB0模型
print("\nEfficientNetB0:\n")
# 评估原始模型在验证集上的性能
evaluate_model(EfficientNetB0, validation_dataset)
# 评估量化后的TFLite模型在验证集上的性能
evaluate_tflite_model(EfficientNetB0_tflite, validation_dataset)
# 将量化后的TFLite模型保存到文件
save_model(EfficientNetB0_tflite, "EfficientNetB0.tflite")

# 评估并保存ResNet101模型
print("\nResNet101:\n")
# 评估原始模型在验证集上的性能
evaluate_model(ResNet101, validation_dataset)
# 评估量化后的TFLite模型在验证集上的性能
evaluate_tflite_model(ResNet101_tflite, validation_dataset)
# 将量化后的TFLite模型保存到文件
save_model(ResNet101_tflite, "ResNet101.tflite")
